# Extraction Unifier

**This tutorial shows how to copy multiple separate extraction jobs into one extraction.**

**To do so we first need to connect to the source data model.**

In [1]:
from pycelonis import get_celonis

c_source = get_celonis("URL to the team from which you want to copy.", "Specify a valid API key for your source cloud team.")
source_data_pool = c_source.pools.find("Name or ID of the source data pool.")
source_data_job = source_data_pool.data_jobs.find("Name or ID of the source data job.")

2020-04-02 08:25:56 - pycelonis: Login successful! Hello Application Key, this key currently has access to 0 analyses.


In [ ]:
for source_extraction in source_data_job.extractions.data: 
    print(source_extraction.name)

**In the next step we connect to the target data pool.**

In [4]:
c_target = get_celonis("URL to cloud team where you want to copy to.", "Specify a valid API key for your target cloud team.")
target_data_pool = c_target.pools.find("Name or ID of the target data pool.")
target_data_job = target_data_pool.data_jobs.find("Name or ID of the target data job.")
target_extraction = target_data_job.extractions.ids["Name or ID of the target data extraction."]

2020-04-02 08:29:22 - pycelonis: Login successful! Hello Application Key, this key currently has access to 0 analyses.


**Create target extraction.**

In [6]:
for source_extraction in source_data_job.extractions.data:   # loop through source extractions

    for source_ex_para in source_extraction.variables:                                                                              # loop through the source extraction parameters
        if source_ex_para['defaultSettings'] is not None:                                                                           # create the connection for the default value to the target global parameters
            if source_ex_para['defaultSettings']['poolVariableId'] is not None:
                target_id = global_vars.get(source_ex_para['defaultSettings']['poolVariableId'])              
                if target_id is None:
                    para = target_data_pool.create_pool_parameter(source_data_pool.variables.find(source_ex_para['defaultSettings']['poolVariableId'])) # create the target global parameter if it does not exist yet
                    print("Pool Parameter '" + para.name + "' created.")
                    target_id = para.id
                    global_vars[source_ex_para['defaultSettings']['poolVariableId']] = target_id
                source_ex_para['defaultSettings']['poolVariableId'] = target_id
        if source_ex_para['settings'] is not None:                                                                                   # create the connection for the value to the target global parameters
            if source_ex_para['settings']['poolVariableId'] is not None:
                target_id = global_vars.get(source_ex_para['settings']['poolVariableId'])
                if target_id is None:
                    para = target_data_pool.create_pool_parameter(source_data_pool.variables.find(source_ex_para['settings']['poolVariableId']))        # create the target global parameter if it does not exist yet
                    print("Pool Parameter '" + para.name + "' created.")
                    target_id = para.id
                    global_vars[source_ex_para['settings']['poolVariableId']] = target_id
                source_ex_para['settings']['poolVariableId'] = target_id

        target_extraction.create_extraction_parameter(source_ex_para)                                                                # create the target transformation parameter
        print("Parameter '" + source_ex_para['name'] + "' created.")
    
    for table in source_extraction.tables.data:

        if table.data['creationDateParameterStart'] is not None or table.data['creationDateParameterEnd'] is not None or table.data['changeDateOffsetParameter'] is not None:

            temp = table.data.copy()

            for para in source_extraction.variables:
                if para['id'] == table.data['creationDateParameterStart']:
                    placeholder_start = para['placeholder']
                if para['id'] == table.data['creationDateParameterEnd']:
                    placeholder_end = para['placeholder']
                if para['id'] == table.data['changeDateOffsetParameter']:
                    placeholder_change = para['placeholder']

            for para in target_extraction.variables:
                if para['placeholder'] == placeholder_start:
                    temp['creationDateParameterStart'] = para['id']
                if para['placeholder'] == placeholder_end:
                    temp['creationDateParameterEnd'] = para['id']
                if para['placeholder'] == placeholder_change:
                    temp['changeDateOffsetParameter'] = para['id']

            table=temp          

        try:
            target_extraction.add_table(table=table)
        except:
            print("Filters for table: '" + table.name + "' are neglected as the table is not found in the indicated source connection.")

    print("Congrats you copied the extraction "+ target_extraction.name + "!")   

Congrats you copied the extraction test_unified!
Congrats you copied the extraction test_unified!
Congrats you copied the extraction test_unified!
Congrats you copied the extraction test_unified!
Parameter 'Maximal Activity End Date' created.
Parameter 'Maximal Activity Start Date' created.
Congrats you copied the extraction test_unified!
Congrats you copied the extraction test_unified!
Congrats you copied the extraction test_unified!
Congrats you copied the extraction test_unified!
Congrats you copied the extraction test_unified!
